<a href="https://colab.research.google.com/github/n-vit/YaP_Projects/blob/main/encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [ ]:
# libraries and functions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from numpy import linalg as LA

# models
from sklearn.linear_model import LinearRegression

# metrics
from sklearn.metrics import r2_score


# Защита персональных данных клиентов

Цель работы - защитить данные клиентов страховой компании «Хоть потоп». 

Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. И обосновать корректность работы выбранного метода. 

Нам нужно защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [ ]:
try: # if the project is opened in Collab use a 'try' block, if it opened in simulator Yandex, or local Jupiter, use an 'except' block
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

except:
    df = pd.read_csv('/datasets/insurance.csv')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.head(3)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


In [ ]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

Данные содержат таблицу из пяти столбцов и 5000 строк с информацией в числовых данных:
- пол(бинарный)
- возраст (вещественные числа)
- зарплата (вещественные числа)
- члены семьи (количество)
- страховые выплаты (количество)

## Умножение матриц

Выделим из фрейма таргет, признаки для обучения модели и тестовую выборку:

In [ ]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.25, 
                                                                            random_state=5)
print(len(features.axes[1]))

4


Создадим обратимую матрицу, с шириной равной количеству признаков для обучения модели:

In [ ]:
def revers_matix(n):
  # функция принимает ширину матрицы, создает случайную обратимую матрицу и проверяет ее на обратимость
    try:
        rand_mx = np.random.normal(size=(n, n))
        LA.inv(rand_mx)
    except LA.LinAlgError: reversible_matix
 
    return rand_mx

In [ ]:
A = revers_matix(len(features.axes[1]))
display(A)

array([[ 0.42046636, -0.55854394,  1.52984532,  1.15817314],
       [ 0.31472006, -2.48320208, -0.73872269,  1.85762646],
       [ 1.30476914, -0.65836082, -0.83625948,  0.38670989],
       [ 1.1930489 , -1.61560762, -1.55872646,  1.1116013 ]])

Зашифруем признаки на обучающей и тестовой выборках, умножив их на обратимую матрицу:

In [ ]:
features_train_encrypt  = features_train @ A
features_test_encrypt = features_test @ A

In [ ]:
len(features_train_encrypt[0].unique())

3675

нда. теперь у нас небинарный пол. да и не пол вовсе


Обучим модель линейной регрессии на незашифрованных даннных и оценим коэффициент детерминации (R2) :


In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print("r2 чистый:", r2_score(target_test, predictions))

r2 чистый: 0.42937359840756484


Теперь обучим ту же модель на зашифрованных данных и сравним полученные результаты:


In [ ]:
model.fit(features_train_encrypt, target_train)
predictions_encrypt = model.predict(features_test_encrypt)
print("r2 на зашифрованных данных:", r2_score(target_test, predictions_encrypt))

r2 на зашифрованных данных: 0.42937359840748346


In [ ]:
print('Разница в предсказаниях модели, обученной на зашифрованных и незашифрованных данных:', r2_score(target_test, predictions) -  r2_score(target_test, predictions_encrypt))

Разница в предсказаниях модели, обученной на зашифрованных и незашифрованных данных: 8.137934770502397e-14


Разница в предсказаниях двух моделей ничтожна

**Ответ:** Шифрование исходных признаков путем умножения их значений на обратимую матрицу - **не влияет на качество предсказаний линейной регрессии** 

**Обоснование:** Для шифрования мы используем умножение матриц. 
Зашифрованная матрица представляет собой набор скалярных произведений векторов:

вектор каждой строки признаков последовательно умножается на вектор каждого столбца обратимой матрицы. 

В нашем случае - значение признака в каждой строке последовательно умножается на фиксированное случайное число и суммируется со следующим произведением значения признака на фиксированное число. Одни и те же множители обратимой матрицы повторяются для каждой строки, а значит **сохраняются зависимости строк друг от друга**, и от таргета, которые и ищет линейная регрессия. 

Практика подтверждает теорию



## Алгоритм преобразования

**Алгоритм** (по сути мы уже выполнили нужное шифрование в предыдущем пункте)
- создаем обратимую матрицу из случайных чисел, с шириной равной количеству столбцов в данных для шифрования
- умножаем незашифрованный фрейм на созданную матрицу - получаем защищенные (зашифрованные данные). Не имея в распоряжении матрицы-множителя мы не сможем ими воспользоваться
- шифрование не влияет на предсказания линейной регрессии, поэтому мы можем обучить на имеющихся данных модель и предсказать новые таргеты по новым признакам
- восстанавливаем данные, умножив их на матрицу, обратную использованной при шифровании

**Обоснование**

Умножение матрицы на обратную ей матрицу дает в результате единичную матрицу той же ширины

Умножение любой матрицы на единичную мартицу возвращает ту же матрицу, поэтому при дешифровании по сути мы "схлопнем" шифр в единичную матрицу

## Проверка алгоритма

Посмотрим на исходные признаки:

In [ ]:
features_train.head(3)

,Пол,Возраст,Зарплата,Члены семьи
4081,1,21.0,45800.0,0
1714,1,43.0,40500.0,1
812,1,37.0,35400.0,1


Вот так они выглядят зашифрованными:


In [ ]:
features_train_encrypt.head(3)

,0,1,2,3
4081,59765.456195,-30205.631286,-38314.667344,17751.481515
1714,52858.296644,-26772.565000,-33900.302746,15743.898456
812,46202.085710,-23400.025612,-29630.947081,13760.532233


индексы те же, а о чем тут - непонятно. 
теперь расшифруем признаки, которые мы зашифровали, умножив на обратную матрицу:

In [ ]:
features_train_decrypt  = (features_train_encrypt @ np.linalg.inv(A)).astype(int)
features_train_decrypt.head(3) 

,0,1,2,3
4081,0,20,45800,0
1714,0,43,40499,0
812,0,37,35400,0


При каждом запуске результат после дешифровки чуть различается (то рубля в доходе не хватает, то года в возрасте). но в целом понятно кто есть кто. 

Метод работает. Можно зашифровать данные клиентов (признаки), предсказать по ним новые целевые признаки, и никто ничего не узнает. 



## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования